# Spectrum

In [1]:
import pandas as pd
import astropy.units as u
import numpy as np
import re
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from pandas.plotting import scatter_matrix

In [2]:
catalog = pd.read_csv("markeddata/gevtevsimbad_TeVspectrum.txt", sep='\t', index_col=0)

In [3]:
catalog.head()

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,marked,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV
0,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128221e-17
1,92,b' ',b' ',b' ',b' ',b'HESS J1841-055 ',b' ',b' ',b' ',b' ',...,b'TeV J1840-055',125,b'TeV J1840-0533',b'gal',0,6.310589e-11,1.207092e-11,8.365091e-13,4.356098e-14,2.865129e-15
2,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128221e-17
3,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,b'TeV J1746-289',-9223372036854775808,b'',b'gal',0,1.647138e-11,8.320850e-14,2.778103e-14,1.856180e-15,3.184437e-16
4,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128221e-17


Columns contained spectrum in gev:

In [4]:
gev_spectrum_columns = [
    'gev_nuFnu10000_100000',
    'gev_nuFnu1000_3000',
    'gev_nuFnu100_300',
    'gev_nuFnu3000_10000',
    'gev_nuFnu300_1000',
    'gev_nuFnu30_100',
]

Columns contained spectrum in simbad:

In [5]:
s_spectrum_columns = [
    's_FLUX_U', 
    's_FLUX_B', 
    's_FLUX_V', 
    's_FLUX_R', 
    's_FLUX_I',
    's_FLUX_G', 
    's_FLUX_J', 
    's_FLUX_H',
    's_FLUX_K', 
    's_FLUX_u',
    's_FLUX_g', 
    's_FLUX_r',
    's_FLUX_i',    
    's_FLUX_z',
    ]

Columns contained spectrum in tev:

In [6]:
tev_spectrum_columns =  [
    'tev_0.3TeV', 
    'tev_1TeV', 
    'tev_3TeV',  
    'tev_10TeV',  
    'tev_30TeV' 
]

In [7]:
s_other_columns = ['s_MAIN_ID',
                 's_class', 
                 ]

In [8]:
gev_other_columns = ['gev_1FGL_Name',
                     'gev_CLASS1',
                    ]

In [9]:
tev_other_columns = ['tev_fermi_names', 
                     'tev_classes',
                    ]

In [10]:
spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns
other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']

In [11]:
spectrums = catalog[spectrum_columns + other_columns].copy()

The next cell computes simbad spectrum using this formula:
$$ F_{simbad} = 10^{-flux_{simbad}}$$

In [12]:
spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])

The next cell computes TeV spectrum using:
$$F_{TeV} = 1.6*E^{2}*Flux$$

In [13]:
for i in tev_spectrum_columns:
    if len(re.findall("\d+", i)) == 2:
        E = 0.3
    else:
        E = int(re.findall("\d+", i)[0])
    spectrums[i] = 1.6 * E ** 2 * spectrums[i]

In [14]:
spectrums = spectrums.fillna(0)

In [15]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,tev_3TeV,tev_10TeV,tev_30TeV,s_MAIN_ID,s_class,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.633748e-12,8.977128e-13,8.824638e-14,b'SSTGC 519103',Be,b' ',b' ',b'',b'unid',0
1,2.098420e-11,2.096862e-11,1.266728e-12,2.274123e-11,7.566604e-12,0.0,0.000000e+00,6.918312e-12,3.090294e-11,0.000000e+00,...,1.204573e-11,6.969757e-12,4.125786e-12,b'LS IV -05 11',B,b' ',b'PWN ',b'',b'unid',0
2,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.633748e-12,8.977128e-13,8.824638e-14,b'GCIRS 1W',B,b' ',b' ',b'',b'unid',0
3,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.000469e-13,2.969888e-13,4.585589e-13,b'[MCD2010] 18',O,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.633748e-12,8.977128e-13,8.824638e-14,b'GCIRS 8',O,b' ',b' ',b'',b'unid',0
5,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,b'[MCD2010] 13',O,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.000469e-13,2.969888e-13,4.585589e-13,b'[MCD2010] 13',O,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,7.585778e-14,8.317639e-14,1.122019e-12,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,b'[MT91] 213',B0Ve,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,4.073800e-13,9.772367e-13,5.011870e-12,8.912505e-12,...,0.000000e+00,0.000000e+00,0.000000e+00,b'Schulte 21',B0Ve,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,5.929003e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,0.0,5.370312e-12,2.454711e-12,1.778279e-11,6.456549e-11,...,8.533807e-13,2.910345e-13,4.065659e-13,b'LS I +61 303',B0Ve,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


Normalization:

In [16]:
# not OK
spectrums[spectrum_columns].div(spectrums.sum(axis=1), axis=0).sum(axis=1)

0     1.000000e+00
1     1.000000e+00
2     1.000000e+00
3     1.000000e+00
4     1.000000e+00
5     1.000000e+00
6     1.000000e+00
7     1.848041e-09
8     1.000000e+00
9     2.123628e-08
10    1.000000e+00
11    1.000000e+00
12    1.000000e+00
13    1.000000e+00
14    1.000000e+00
15    1.000000e+00
16    1.000000e+00
17    1.000000e+00
18    1.000000e+00
19    1.000000e+00
20    1.000000e+00
21    1.000000e+00
22    1.000000e+00
23    1.000000e+00
24    1.000000e+00
25    1.000000e+00
26    1.000000e+00
27    1.000000e+00
28    1.000000e+00
29    1.000000e+00
          ...     
38    2.185807e-10
39    1.000000e+00
40    1.000000e+00
41    1.000000e+00
42    1.000000e+00
43    1.000000e+00
44    1.000000e+00
45    1.000000e+00
46    1.000000e+00
47    1.000000e+00
48    1.000000e+00
49    1.000000e+00
50    1.000000e+00
51    1.000000e+00
52    1.000000e+00
53    1.000000e+00
54    1.000000e+00
55    1.000000e+00
56    1.000000e+00
57    1.000000e+00
58    1.000000e+00
59    1.0000

In [18]:
# OK
spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0).sum(axis=1)

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
     ... 
38    1.0
39    1.0
40    1.0
41    1.0
42    1.0
43    1.0
44    1.0
45    1.0
46    1.0
47    1.0
48    1.0
49    1.0
50    1.0
51    1.0
52    1.0
53    1.0
54    1.0
55    1.0
56    1.0
57    1.0
58    1.0
59    1.0
60    1.0
61    1.0
62    1.0
63    1.0
64    1.0
65    1.0
66    1.0
67    1.0
Length: 68, dtype: float64

In [19]:
spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)

The cell displays all information we have about spectrums now.

In [20]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,tev_3TeV,tev_10TeV,tev_30TeV,s_MAIN_ID,s_class,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,0.013537,0.343081,0.020826,0.143005,0.330419,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.012135,0.004136,0.000407,b'SSTGC 519103',Be,b' ',b' ',b'',b'unid',0
1,0.013335,0.013325,0.000805,0.014451,0.004808,0.0,0.000000e+00,4.396364e-03,1.963782e-02,0.000000,...,0.007655,0.004429,0.002622,b'LS IV -05 11',B,b' ',b'PWN ',b'',b'unid',0
2,0.001682,0.042624,0.002587,0.017767,0.041051,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.001508,0.000514,0.000051,b'GCIRS 1W',B,b' ',b' ',b'',b'unid',0
3,0.011399,0.129145,0.001246,0.070802,0.082033,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000969,0.000719,0.001111,b'[MCD2010] 18',O,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,0.012809,0.324638,0.019706,0.135318,0.312657,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.011482,0.003914,0.000385,b'GCIRS 8',O,b' ',b' ',b'',b'unid',0
5,0.005245,0.059427,0.000573,0.032580,0.037748,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,b'[MCD2010] 13',O,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,0.005224,0.059186,0.000571,0.032448,0.037595,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000444,0.000330,0.000509,b'[MCD2010] 13',O,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,0.001540,0.024893,0.007678,0.017450,0.017155,0.0,4.104766e-05,4.500786e-05,6.071395e-04,0.000000,...,0.000000,0.000000,0.000000,b'[MT91] 213',B0Ve,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,0.000752,0.012156,0.003749,0.008521,0.008377,0.0,1.076456e-04,2.582238e-04,1.324330e-03,0.002355,...,0.000000,0.000000,0.000000,b'Schulte 21',B0Ve,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,0.000279,0.004163,0.006375,0.001910,0.006206,0.0,2.528838e-04,1.155904e-04,8.373780e-04,0.003040,...,0.000040,0.000014,0.000019,b'LS I +61 303',B0Ve,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


In [21]:
spectrums.to_csv("markeddata/normalized_spectra.txt", index=False)

In [22]:
def calculate_spectra(catalog_name, s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns,
                      s_other_columns, gev_other_columns, tev_other_columns):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    if len(s_spectrum_columns) > 0:
        spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])
    for i in tev_spectrum_columns:
        if len(re.findall("\d+", i)) == 2:
            E = 0.3
        else:
            E = int(re.findall("\d+", i)[0])
        spectrums[i] = 1.6 * E ** 2 * spectrums[i]
    spectrums = spectrums.fillna(0)
    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)
    spectrums = spectrums.fillna(0)
    display(spectrums)
    spectrums.to_csv("markeddata/" + catalog_name + "_normalized.txt", index=False)

In [23]:
calculate_spectra(catalog_name="gevtev_TeVspectrum", 
                  s_spectrum_columns=[], 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=tev_spectrum_columns, 
                  s_other_columns=[],
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="tevsimbad_TeVspectrum", 
                  s_spectrum_columns =s_spectrum_columns, 
                  gev_spectrum_columns=[], 
                  tev_spectrum_columns=tev_spectrum_columns, 
                  s_other_columns=s_other_columns,
                  gev_other_columns=[],
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="gevsimbad", 
                  s_spectrum_columns=s_spectrum_columns, 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=[], 
                  s_other_columns=s_other_columns,
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=[])

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,0.055155,0.201477,0.000004,0.235066,0.028249,0.0,0.391619,0.072840,0.013193,2.028336e-03,3.673713e-04,b' ',b'bll ',b'',b'hbl',0
1,0.069119,0.073172,0.003407,0.038325,0.145615,0.0,0.134071,0.020429,0.006452,5.937939e-02,4.500310e-01,b' ',b'snr ',b'',b'snr',0
2,0.358190,0.178179,0.096737,0.268908,0.097987,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,b'1FGL J0033.5-1921 ',b'bll ',b'',b'hbl',0
3,0.269005,0.157462,0.139724,0.220561,0.213247,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,b'1FGL J0035.9+5951 ',b'bll ',b'',b'hbl',0
4,0.427725,0.147237,0.104148,0.231066,0.089825,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,b'1FGL J0136.5+3905 ',b'bll ',b'',b'hbl',0
5,0.184803,0.139294,0.092967,0.072162,0.064664,0.0,0.270907,0.072492,0.049008,3.200621e-02,2.169678e-02,b' ',b'bll ',b'',b'hbl',0
6,0.006631,0.235676,0.352788,0.087054,0.317851,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,b'1FGL J0205.6+6449 ',b'PSR ',b'',b'pwn',0
7,0.080013,0.167359,0.363962,0.139325,0.249341,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,b'1FGL J0221.0+3555 ',b'FSRQ ',b'',b'blazar',0
8,0.193215,0.210747,0.145323,0.230057,0.192865,0.0,0.021036,0.005001,0.001348,3.205639e-04,8.642315e-05,b'1FGL J0222.6+4302 ',b'BLL ',b'',b'ibl',0
9,0.198738,0.216772,0.149478,0.236634,0.198378,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,b'1FGL J0222.6+4302 ',b'BLL ',b'',b'unid',0


,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,s_FLUX_I,s_FLUX_G,s_FLUX_J,s_FLUX_H,s_FLUX_K,s_FLUX_u,...,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,s_MAIN_ID,s_class,tev_fermi_names,tev_classes,marked
0,0.000000,0.000000,0.000000,0.000060,0.000000,0.000143,1.701847e-03,0.003243,0.007779,0.0,...,0.484812,0.272675,0.056902,0.037315,0.135368,b'UCAC4 105-013663',Be,b'',b'other',0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.376672e-04,0.019565,0.869940,0.0,...,0.045760,0.029010,0.025940,0.008455,0.000992,b'MSX6C G331.5487-00.5364',Be,b'',b'unid',0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.607538e-06,0.002454,0.616424,0.0,...,0.148664,0.120621,0.081368,0.027734,0.002726,b'SSTGC 519103',Be,b'',b'unid',0
3,0.000000,0.027707,0.045983,0.000000,0.000000,0.000000,1.326154e-01,0.170842,0.205397,0.0,...,0.003439,0.230194,0.136930,0.045784,0.001110,b'RMC 141',B,b'',b'pwn',0
4,0.000000,0.000531,0.000618,0.000343,0.000474,0.000618,3.846459e-04,0.000260,0.000244,0.0,...,0.489456,0.275287,0.057447,0.037672,0.136665,b'[AAA97b] J053648.7-691700',B,b'',b'other',0
5,0.000000,0.000018,0.000022,0.000022,0.000042,0.000033,1.555617e-05,0.000016,0.000015,0.0,...,0.008237,0.551319,0.327949,0.109654,0.002657,b'OGLE LMC-ECL-20994',B,b'',b'pwn',0
6,0.009568,0.001909,0.001625,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,...,0.484726,0.272627,0.056892,0.037308,0.135344,b'[ST92] 2-62B',B,b'',b'other',0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.518918e-03,0.033116,0.085711,0.0,...,0.383273,0.292811,0.142309,0.043507,0.014754,b'[KKM2013] 7',B,b'',"b'pwn,snr'",0
8,0.004387,0.001871,0.026432,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,...,0.475105,0.267216,0.055762,0.036568,0.132658,b'[ST92] 2-62',B,b'',b'other',0
9,0.002968,0.000958,0.003797,0.009895,0.452295,0.004114,8.154943e-02,0.157551,0.250277,0.0,...,0.000301,0.020180,0.012004,0.004014,0.000097,b'HTR 13',B,b'',b'pwn',0


,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,s_FLUX_u,s_FLUX_g,s_FLUX_r,s_FLUX_i,s_FLUX_z,s_MAIN_ID,s_class,gev_1FGL_Name,gev_CLASS1,marked
0,1.656470e-02,0.318116,1.167143e-01,0.194982,3.536225e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[KRL2007b] 194',Be,b'1FGL J1636.4-4737c',b'snr ',0
1,4.367979e-06,0.000080,1.744995e-05,0.000040,7.974043e-05,0.0,8.700521e-05,3.626980e-05,8.119804e-05,1.022224e-04,...,0.000000e+00,4.360598e-05,1.228983e-04,1.411059e-04,0.000000e+00,b'HD 328990',Be,b'1FGL J1651.5-4602c',b' ',0
2,1.435131e-02,0.087269,9.648044e-02,0.058873,8.264021e-02,0.0,4.192119e-05,2.251299e-05,1.788273e-04,8.173980e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'UCAC2 5051621',Be,b' ',b'spp ',0
3,5.414631e-02,0.123705,4.429596e-01,0.121311,1.887937e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'SSTGC 519103',Be,b'1FGL J1745.6-2900c',b'spp ',0
4,4.642829e-02,0.232212,2.710111e-01,0.145261,3.043594e-01,0.0,0.000000e+00,2.831540e-05,2.624356e-05,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'LIN 341',Be,b'1FGL J0101.3-7257 ',b'GAL ',0
5,1.523725e-03,0.006400,1.623092e-02,0.004016,8.842059e-03,0.0,1.508756e-01,3.456357e-02,5.231411e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'LS VI -04 15',B,b' ',b' ',0
6,1.573251e-04,0.002620,8.954646e-03,0.001090,3.881067e-03,0.0,0.000000e+00,6.987534e-03,1.528706e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'LS I +56 58',B,b' ',b' ',0
7,1.435295e-05,0.131313,4.130346e-01,0.074241,3.705391e-01,0.0,0.000000e+00,2.135673e-04,4.462050e-04,3.158892e-04,...,2.200565e-04,4.915121e-04,4.748264e-04,4.173824e-04,3.503764e-04,b'PG 1101+529',B,b' ',b' ',0
8,1.144422e-05,0.240199,3.359969e-01,0.083138,3.247328e-01,0.0,0.000000e+00,0.000000e+00,1.592146e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'GALEX J175340.5-500741',B,b'1FGL J1754.0-5002 ',b'bcu ',0
9,1.941257e-02,0.118046,1.305061e-01,0.079635,1.117848e-01,0.0,2.011987e-03,2.058849e-03,7.649360e-03,1.457555e-02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'HD 306182',B,b' ',b'spp ',0
